In [ ]:
#IMPORT LIBRARIES REQUIRED FOR THE CODE

%matplotlib inline

import csv
import os
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import re
import sys
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.externals import joblib

In [ ]:
# READ THE CRAWLED CSV FILES AND COMBINE THEM INTO ONE ARRAY

def get_field_names(paths):
    """
    Given a list of paths to csv files,
    this function will find all the column header names
    and return them in the order encountered.
    """
    field_set = set()
    fields = []
    
    for path in paths:
        with open(path, 'r') as csvfile:
            reader = csv.DictReader(csvfile, delimiter=',')
            
            for field in reader.fieldnames:
                #for every field that we just encountered,
                
                if field not in field_set:
                    #if we have not yet seen this field,
                    
                    #add the field to the (in order) list
                    fields.append(field)
                    
                    #mark the field as "seen"
                    field_set.add(field)
    return fields


def get_relevant_csv_filenames(base_path):
    """
    Given a base path, this function will go through
    every file in that directory, and it will return the
    files that end with the .csv extension, as a list.
    """
    
    paths = []
    for filename in os.listdir(base_path):
        _,extension = os.path.splitext(filename)

        if extension != '.csv':
            continue
        paths.append(filename)
    
    return paths
    
    
paths = get_relevant_csv_filenames(base_path='../')
paths=['../'+s for s in paths]

field_names = get_field_names(paths)
print ('field_names:',field_names)


data = []

for filename in paths:
    base,_ = os.path.splitext(filename)
    
    _,_,year = base.partition('_')
    
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',')
        
        for row in reader:
            row['year'] = year
            data.append(row)
            
            #print (row)
            #raise Exception()

#print (len(data))

#Convert the aggregate data into pandas dataframe
df=pd.DataFrame(data)
print(list(df.columns))

# Split age_sex column into two columns of age and sex
df['sex']=list(list(zip(*[re.findall("[a-zA-Z]+",i) for i in list(df.sex_age)]))[0])
ages=list(list(zip(*[re.findall("[0-9]+",i) for i in list(df.sex_age)]))[0])
ages = [int(age) for age in ages]
df['age'] = ages

In [ ]:
# PRE-PROCESS DATA: BASIC CLEAN-UP + HANDLE MISSING DATA

# Drop columns that are not necessary and inconsistent among time series data
df.drop(df.columns[[9,11,13,17,18,22,24]], axis=1,inplace=True) # Note: zero indexed

print(list(df.columns))

# Remove rows that have missing values
df['state'].replace('','Missing',inplace=True)

# Find countries that are not United States, and replace the values of corresponding state
# Missing
cntry_id=[cntry_name!='United States' for cntry_name in df.country]
cntry_id=[i for i, x in enumerate(cntry_id) if x] # get indictes

df.loc[cntry_id,'state']='Missing'

df[:].replace('', np.nan, inplace=True) # Replacing empty strings with NaN

#append _ before all column names to avoid problems
df.columns='Var_'+df.columns 

In [ ]:
# STORE THE PROCESSED DATA IN SQL DATABASE


# Retain all data including missing elements as those rows are needed later
# to find people who ran the marathon multiple times (experience)
conn = sqlite3.connect('marathon_runner_data.sqlite')
conn.text_factory=str

c = conn.cursor()

sql_columns = '\n  , '.join( [('"%s" text' % (field,)) for field in df.columns] )

table_sql = """

CREATE TABLE runner_data
(
    {columns}
);
"""

table_sql = table_sql.format(columns=sql_columns)

print (table_sql);
c.execute("DROP TABLE IF EXISTS runner_data;")
c.execute(table_sql)

  
for (index,row_series) in df.iterrows():
    
    sql = """
    INSERT INTO runner_data ({columns_list})
    VALUES ({values})
    """
    columns_list = ', '.join( [('"%s"' % (field,)) for field in df.columns] )

    #the values should look like this (?,?,?,?,?, ...)
    values_question_marks = ['?']*len(row_series)
    values_question_marks = ', '.join(values_question_marks)
    
    sql = sql.format(columns_list=columns_list, 
                     values=values_question_marks)
        
    #print(sql) 
    c.execute(sql, tuple(row_series))
#    raise Exception()
conn.commit()
conn.close()

In [ ]:
# READ SQL DATABASE AND RUN QUERIES TO CALCULATE "PRIOR EXPERIENCE"

conn = sqlite3.connect('marathon_runner_data.sqlite')
conn.text_factory=str

c = conn.cursor()

# Add a column named "experience" that says how many times each person has run the NYC marathon 
# and pull out records of people who have run more than once
c.execute("DROP TABLE IF EXISTS temporary_table")

sql = '''
CREATE TABLE temporary_table AS
SELECT runner_data.*,experience
FROM runner_data
JOIN (SELECT Var_first_name,Var_last_name,Var_sex,Var_state,Var_country,
Var_age,Var_year,count(*) as experience
FROM runner_data
GROUP BY Var_first_name,Var_last_name,Var_sex,Var_state,Var_country
HAVING count(*)>1) as TEMP

ON runner_data.Var_first_name=TEMP.Var_first_name
AND runner_data.Var_last_name=TEMP.Var_last_name
AND runner_data.Var_sex=TEMP.Var_sex
AND runner_data.Var_country=TEMP.Var_country
AND runner_data.Var_state=TEMP.Var_state
AND runner_data.Var_age<=TEMP.Var_age
AND runner_data.Var_year<TEMP.Var_year
AND ((TEMP.Var_year-runner_data.Var_year)-(TEMP.Var_age-runner_data.Var_age))<2

'''

c.execute(sql)
#len(c.fetchall())

c.execute("DROP TABLE IF EXISTS runner_data")

sql='''
ALTER TABLE temporary_table 
RENAME TO runner_data
'''
c.execute(sql)

# Clear memory
c.execute("DROP TABLE IF EXISTS temporary_table")

# Eliminate data prior to 2006

sql= '''
DELETE FROM runner_data
WHERE Var_year<2006
'''
c.execute(sql)

In [ ]:
# GET THE SQL DATABASE INTO DATAFRAME

c.execute("SELECT * FROM runner_data")
df = pd.DataFrame(c.fetchall())
df.columns =[description[0] for description in c.description]

# Delete columns you do not require
df.drop(['Var_country','Var_first_name','Var_last_name','Var_state',
        'Var_age_graded_time','Var_place','Var_place_age',
         'Var_place_gender'], axis=1, inplace=True)

In [ ]:
# DELETE ROWS WITH NAN VALUES
df= df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False); 

In [ ]:
# ELIMINATE ROWS WHERE TIME HAS FORMATTING ISSUES

col_names=['Var_5km','Var_10km','Var_15km','Var_20km','Var_25km','Var_30km',
          'Var_35km','Var_40km','Var_13.1mi','Var_gun_time']

for variable in col_names:
    typo_time=[len(i) for i in df[variable]]
    typo_id=[i for i, x in enumerate(typo_time) if x is not 7] # get indictes
    df=df.drop(df.index[typo_id])

In [ ]:
# CONVERT ALL TIME VARIABLES FORMATES TO TOTAL MINUTES

col_names=['Var_5km','Var_10km','Var_15km','Var_20km','Var_25km','Var_30km',
          'Var_35km','Var_40km','Var_13.1mi','Var_gun_time']

for col_name in col_names:
    #get a list of all the values in column col_name
    column_values0 = df.loc[0:][col_name]
    
    #this is going to hold a new list of transformed
    # values
    column_values1 = []
    
    #for each value in the column
    for value0 in column_values0:
        #split it into hours,minutes,seconds
        #print(value0)

        HH,MM,SS = value0.split(':')
        
        #compute the float total of minutes
        value1 = float(HH)*60 + float(MM) + float(SS)/60
        
        #append it to the new list of column-values
        column_values1 += [value1]
    #the loop has finished, we now have the new list of column values
    
    #therefore assign back to the table
    df[col_name] = column_values1

In [ ]:
# Convert ALL DATA TYPES TO FLOAT

datatype_convert=['Var_year','Var_age']
df[datatype_convert] = df[datatype_convert].astype(float)
datatype_convert=[];

# Create Y vector
col_names=['Var_5km','Var_10km','Var_15km','Var_20km','Var_25km','Var_30km',
          'Var_35km','Var_40km','Var_13.1mi']

Y=df[col_names]

# Drop col_names from df
df.drop(col_names, axis=1, inplace=True)

In [ ]:
# CONVERT CUMULATIVE SPLIT TIMES INTO TIME IN EACH SPLIT

pd.options.mode.chained_assignment = None  # default='warn'

Y.loc[:,'Var_Finish']=df['Var_gun_time']-Y['Var_40km']
Y.loc[:,'Var_40km']=Y['Var_40km']-Y['Var_35km']
Y.loc[:,'Var_35km']=Y['Var_35km']-Y['Var_30km']
Y.loc[:,'Var_30km']=Y['Var_30km']-Y['Var_25km']
Y.loc[:,'Var_25km']=Y['Var_25km']-Y['Var_13.1mi']
Y.loc[:,'Var_13.1mi']=Y['Var_13.1mi']-Y['Var_20km']
Y.loc[:,'Var_20km']=Y['Var_20km']-Y['Var_15km']
Y.loc[:,'Var_15km']=Y['Var_15km']-Y['Var_10km']
Y.loc[:,'Var_10km']=Y['Var_10km']-Y['Var_5km']

In [ ]:
# CONVERT RESPONSE DATA TO FRACTIONS

Y=Y.divide(df['Var_gun_time'],axis='index')

In [ ]:
# READ AND STORE THE WEATHER DATA IN SQL DATABASE


# The raw data is in csv format and retrieved from NLDAS data
# from NASA Giovanni interactive database

# Read csv file
data_directory='../Weather_data_raw/'
paths = get_relevant_csv_filenames(base_path=data_directory)
paths=[data_directory+path for path in paths]
field_names = get_field_names(paths)
print ('field_names:',field_names)

data = []

for filename in paths:
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',')
        
        for row in reader:
            data.append(row)
            
            #print (row)
            #raise Exception()

print (len(data))

#Convert the aggregate data into pandas dataframe
wdf=pd.DataFrame(data)
print(list(wdf.columns))

conn = sqlite3.connect('weather_data.sqlite')
conn.text_factory=str

c = conn.cursor()

sql_columns = '\n  , '.join( [('"%s" text' % (field,)) for field in wdf.columns] )

table_sql = """

CREATE TABLE marathon_weather_data
(
    {columns}
);
"""

table_sql = table_sql.format(columns=sql_columns)

print (table_sql);
c.execute("DROP TABLE IF EXISTS marathon_weather_data;")
c.execute(table_sql)

  
for (index,row_series) in wdf.iterrows():
    
    sql = """
    INSERT INTO marathon_weather_data ({columns_list})
    VALUES ({values})
    """
    columns_list = ', '.join( [('"%s"' % (field,)) for field in wdf.columns] )

    #the values should look like this (?,?,?,?,?, ...)
    values_question_marks = ['?']*len(row_series)
    values_question_marks = ', '.join(values_question_marks)
    
    sql = sql.format(columns_list=columns_list, 
                     values=values_question_marks)
        
    #print(sql) 
    c.execute(sql, tuple(row_series))
#    raise Exception()
conn.commit()
conn.close()

In [ ]:
# READ WEATHER DATA AND APPEND AS FEATURES 

conn = sqlite3.connect('weather_data.sqlite')
conn.text_factory=str

d = conn.cursor()

d.execute("SELECT * FROM marathon_weather_data")
wdf = pd.DataFrame(d.fetchall())
wdf.columns =[description[0] for description in d.description]

# Change data types
wdf[wdf.columns] = wdf[wdf.columns].astype(float)

In [ ]:
# Calculate weather conditions for each person based on their timings and year

gun_time_hours=list((df['Var_gun_time']/60))

for i,data in enumerate(gun_time_hours):
    # Get the marathon day's data in a new temporary dataframe 
    # Waves start at 9:45, but adding one hour as they arrive early
    wdf_temp=wdf.loc[wdf['Year'].isin([df.loc[df.index[i],'Var_year']])]
    wdf_temp.loc[wdf_temp['hour'].isin(range(9,11+int(data)))
                                            ,'Rel. Humidity'].mean(axis=0)
    df.loc[df.index[i],'Rel. Humidity']= wdf_temp.loc[wdf_temp['hour'].isin(range(9,11+int(data)))
                                            ,'Rel. Humidity'].mean(axis=0)
    df.loc[df.index[i],'Temperature']= wdf_temp.loc[wdf_temp['hour'].isin(range(9,11+int(data)))
                                            ,'tmp2m'].mean(axis=0)
    df.loc[df.index[i],'X_wind']= wdf_temp.loc[wdf_temp['hour'].isin(range(9,11+int(data)))
                                            ,'ugrdh'].mean(axis=0)
    df.loc[df.index[i],'Y_wind']= wdf_temp.loc[wdf_temp['hour'].isin(range(9,11+int(data)))
                                            ,'vgrd1'].mean(axis=0)

In [ ]:
# Drop unecessary column: Delete year column you no more require

df.drop(['Var_year'], axis=1, inplace=True)

In [ ]:
# Dummy code categorical variables 

dummy_code_var=['Var_sex']
dummy_data= pd.DataFrame()

for dummy_var in dummy_code_var:
    dummies=pd.get_dummies(df[dummy_var])
    dummy_data = pd.concat([dummy_data, dummies], axis=1)
    # Drop the column from df dataframe
    df.drop([dummy_var], inplace=True, axis=1)
    # Avoiding dummy-variable trap
    dummy_data.drop([list(dummies.columns.values)[1]], inplace=True, axis=1)

dummy_data=dummy_data.as_matrix()
df=df.as_matrix()

In [ ]:
# This information is not used in the model. But, is useful for later interpretation.
# These variables relate to course information by splits

'''
# Course conditions (NYC) for each split time
Distance=[5,5,5,5,1.08,3.92,5,5,5,2.2]

Elevation_gain=[178.0382269,69.11468055,96.66463271,62.74108641,41.21141586,
                131.4207563,105.3092448,64.24585234,130.97302,58.67416456]

Elevation_loss=[-197.0382269,-116.7028596,-97.68808331,-106.0469012,
                -42.6731828,-38.04677694,-154.8496651, -72.23193359,
                -49.4847453,-46.70543204]
'''

In [ ]:
# This part of the code is to runa t-SNE algorithm for 
# dimensionality reduction of split time data and 
# visualize the data in relation to experience

import random
import tsne
import pylab as Plot


exp=list(df[:,2])

# Pick random sample with experience range
rand_item = random.sample([i for i,y in enumerate(exp) if y<=2],2000)
rand_item = rand_item+random.sample([i for i,y in enumerate(exp) if y<=8 and y>5],2000)
#rand_item = rand_item+random.sample([i for i,y in enumerate(exp) if y<=5 and y>3],1000)
#rand_item = rand_item+random.sample([i for i,y in enumerate(exp) if y<=10 and y>5],1000)
rand_item = rand_item+random.sample([i for i,y in enumerate(exp) if y>16],2000)

y=Y.as_matrix()[rand_item,:]
y = tsne.tsne(y, 2, 20.0)

In [ ]:
# PLOT DATA FROM t-SNE ANALYSIS

labels=['Newbie','Intermediate','Experienced']#,'3-5 years','5-10 years','>10 years']
colors = ['b', 'g', 'r'] #y', 'm', 'r']

a = plt.scatter(y[0:1999,0], y[0:1999,1], 3, color=colors[0])
b = plt.scatter(y[2000:3999,0], y[2000:3999,1], 3, color=colors[1])
c  = plt.scatter(y[4000:5999,0], y[4000:5999,1], 3, color=colors[2])
#d  = plt.scatter(y[3000:3999,0], y[3000:3999,1], 10, color=colors[3])
#e  = plt.scatter(y[4000:4999,0], y[4000:4999,1], 10, color=colors[4])

plt.legend((a,b,c),tuple(labels),scatterpoints=1,
           loc='lower left',ncol=3, fontsize=10)
plt.title('t-SNE analysis on NYC Marathon Split Times')

plt.savefig('t-SNE_Marathon.png',dpi=1200)
plt.show()

In [ ]:
# Save scaling information for later use

scaler = preprocessing.StandardScaler().fit(df)
joblib.dump(scaler, 'scaling_variables.pkl') 

In [ ]:
# OUTPUT DATA FOR CHECK-POINT AND EASY RETRIEVAL

# Standardize all continuous variables
X = preprocessing.scale(df)

In [ ]:
# Create polynomial features of order 2: Non-linear and interaction effects
poly = PolynomialFeatures(2)
X=poly.fit_transform(X)

In [ ]:
# Merge categorical and continuous data
X=np.concatenate([X,dummy_data],axis=1)

# Response fractional matrix
y=Y.as_matrix()

#Dump the data to csv for easy retrieval..The data is small
np.savetxt("Features_Standardized.csv", X, delimiter=",")
np.savetxt("Response_fractions.csv", y, delimiter=",")

In [ ]:
# Prepare X and y data and local weights for model fitting
# taken out ele loss, prediction si 100%
# when ele gain is taken out the predicrtion is 100%

course_info=np.transpose(np.repeat([Distance,Elevation_gain,Elevation_loss
                                    ],Y.shape[0],axis=1))

X=np.concatenate((np.tile(df,(Y.shape[1],1)),course_info),axis=1)

In [ ]:
# Standardize all continuous variables
X = preprocessing.scale(X)

In [ ]:
# Create polynomial features of order 2: Non-linear and interaction effects
poly = PolynomialFeatures(2)
X=poly.fit_transform(X)

In [ ]:
# Merge categorical and continuous data
X=np.concatenate([X,np.tile(dummy_data,(Y.shape[1],1))],axis=1)

# Flatten y for numpy to interpret correctly
y=np.ravel(np.repeat(range(Y.shape[1]),Y.shape[0]))

weights=np.repeat(np.transpose(Y.as_matrix()),1)

In [ ]:
'''
# Run the multinomial Logistic regression from scikit-learn
# This was just an experiemntation case

model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, 
                           fit_intercept=True, class_weight=None, 
                           random_state=None, solver='lbfgs', max_iter=100000, 
                           multi_class='multinomial', verbose=0, warm_start=False)

model = model.fit(X, y,sample_weight=weights)

# check the accuracy on the training set
model.score(X, y,sample_weight=weights)
'''

In [ ]:
'''
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=3)
print scores
print scores.mean()
'''

In [ ]:
# MACHINE LEARNING ALGORITHM


#Run the Fractional Multinomial Logistic Regression model
#This model is a multinomial generalization of the fractional
#logit model proposed by Papke and Woolridge in 1996

# Requires Python wrapper to Glmnet package available at 
#https://github.com/dwf/glmnet-python
    
    
from glmnet import LogisticNet
from sklearn.datasets import make_classification

display_bar = '-'*70
 
X=df
y=np.ones((len(X),), dtype=np.int)
y[len(y)-1]=0
y[len(y)-2]=2

#y=np.repeat(np.transpose(Y.as_matrix()),1)

print display_bar
print "Fit a logistic net on marathon data"
print display_bar

lognet = LogisticNet(alpha=0.5)
lognet.fit(X, y)

print lognet
 
print display_bar
print "Predictions for the last logistic net model:"
print display_bar

preds = lognet.predict(X)
print preds[:10,np.shape(preds)[1]-1]

lognet.plot_paths()

In [ ]:
# PLOT FREQUENCY DISTRIBUTION OF AVERAGE RUNNING TIME BY AGE AND GENDER

N = len(df.groupby("Year1"))
menMeans = list(df.groupby(["year1","sex"]).mean().gun_time)[1::2]
menStd = list(df.groupby(["year1","sex"]).mean().gun_time)[1::2]

womenMeans = list(df.groupby(["year1","sex"]).mean().gun_time)[::2]
womenStd = list(df.groupby(["year1","sex"]).mean().gun_time)[::2]

ind = np.arange(N)  # the x locations for the groups
width = 0.35       # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, menMeans, width, color='r', yerr=menStd)


rects2 = ax.bar(ind + width, womenMeans, width, color='y', yerr=womenStd)

# add some text for labels, title and axes ticks
ax.set_ylabel('Time in minutes')
ax.set_title('Marathon finish time by gender')
ax.set_xticks(ind + width)
ax.set_xticklabels(tuple(np.unique(df["year1"])))

ax.legend((rects1[0], rects2[0]), ('Men', 'Women'))


def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

plt.show()